In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from metpy.units import units
from metpy import calc as mpcalc
import math
import netCDF4
from netCDF4 import Dataset
from datetime import datetime
from datetime import datetime, timedelta

In [2]:
#Reading in Datasets
t_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/RT_ECMWF_t.nc")
ct_ds = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")

In [3]:
t_ds

<xarray.Dataset>
Dimensions:    (longitude: 240, latitude: 121, level: 10, number: 50, step: 47, date: 8)
Coordinates:
  * longitude  (longitude) float32 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
  * latitude   (latitude) float32 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * level      (level) int32 10 50 100 200 300 500 700 850 925 1000
  * number     (number) int32 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 0 days 1 days 2 days ... 45 days 46 days
  * date       (date) datetime64[ns] 2021-01-04 2021-01-07 ... 2021-01-28
Data variables:
    t          (date, step, number, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-02-23 19:29:30 GMT by grib_to_netcdf-2.20.0: grib_to_n...

In [4]:
ct_ds

<xarray.Dataset>
Dimensions:  (time: 366, hour: 4, lev: 37, lat: 256, lon: 512)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * hour     (hour) float64 0.0 6.0 12.0 18.0
  * lev      (lev) int32 1000 975 950 925 900 875 850 825 ... 30 20 10 7 5 3 2 1
  * lat      (lat) float32 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon      (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Data variables:
    t        (time, hour, lev, lat, lon) float32 ...

In [5]:
# Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Hour Specification
Hour = 0
hourStr = str(Hour)

#Box coordinates: [[49,-108], [49,-87], [32,-87], [32,-108],[49,-108]]

In [6]:
#Working with Climo Data (ERAI)
ct_0z = ct_ds['t'].sel(lev=vlevel, hour=hourStr) #<-- this is climo at 1000mb for only 00z
time = ct_0z.time

In [7]:
ct_0z

<xarray.DataArray 't' (time: 366, lat: 256, lon: 512)>
[47972352 values with dtype=float32]
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
    hour     float64 0.0
    lev      int32 1000
  * lat      (lat) float32 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon      (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Attributes:
    mapping:          cylindrical_equidistant_projection_grid
    grid_resolution:  0.5 degrees
    years_used:       1979-2010
    units:            K
    long_name:        Temperature

In [8]:
#Reindex ct_0z in order to have only the lats and lons we're interested in (will also have to do for t_ensmn
##da.where(da.x + da.y < 4) <-- use this notation (i think)

In [9]:
#Creating Variables with S2S data
lat = t_ds.latitude
lon = t_ds.longitude
level = t_ds.level #<--Levels=[10,50,100,200,300,500,700,850,925,1000]
step = t_ds.step
number = t_ds.number #<--ensemble #
date = t_ds.date
t_0z = t_ds['t'].sel(level=vlevel) #<--forecasted temps at 1000mb

step_in_days = t_ds.step["step.days"] #<-- days forecasted out (47)
forcast_day = t_ds.date["date.dayofyear"] #<--day of forecast(8)


In [10]:
#Calculating Ensemble Mean T for forcasted Temps
#t_ensmn = t_0z.mean(dim='number')

In [12]:
#t_ensmn_ds = t_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF_ensmn_t.nc",
  #                             mode='w')

In [ ]:
t_ensmn_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/RT_ECMWF_t.nc")

In [14]:
#Making data array of each forcast day from day forcasted
for x in forcast_day:
    fc_day_array = forcast_day + step_in_days

In [15]:
ct_0z_slice = ct_0z[3:74,:,:]
ct_series = ct_0z.to_series()
ct_time_series = ct_0z['time'].to_series()
ct_time_list = ct_time_series.tolist()
ct_series #<--multiIndex pandas series


time   lat         lon       
1.0    -89.462822  0.000000      269.361664
                   0.703125      269.362274
                   1.406250      269.362854
                   2.109376      269.363342
                   2.812501      269.363831
                                    ...    
366.0   89.462822  356.484497    247.944534
                   357.187622    247.945984
                   357.890747    247.947418
                   358.593872    247.948868
                   359.296997    247.950363
Name: t, Length: 47972352, dtype: float32

In [18]:
ct_series.tolist() #<-- this no longer has the lon/lats associated with the temps

[269.3616638183594,
 269.3622741699219,
 269.36285400390625,
 269.36334228515625,
 269.36383056640625,
 269.36431884765625,
 269.3646545410156,
 269.3650207519531,
 269.3653869628906,
 269.3656005859375,
 269.3658142089844,
 269.3659973144531,
 269.36614990234375,
 269.36627197265625,
 269.36627197265625,
 269.3663024902344,
 269.36627197265625,
 269.3662414550781,
 269.3660583496094,
 269.3659362792969,
 269.3657531738281,
 269.3655090332031,
 269.3652648925781,
 269.36492919921875,
 269.3645324707031,
 269.3641662597656,
 269.36370849609375,
 269.36328125,
 269.36273193359375,
 269.3622131347656,
 269.36163330078125,
 269.3609924316406,
 269.3603210449219,
 269.3595886230469,
 269.3589172363281,
 269.3581237792969,
 269.3572692871094,
 269.3564453125,
 269.3555908203125,
 269.35467529296875,
 269.353759765625,
 269.3527526855469,
 269.35174560546875,
 269.3507080078125,
 269.3496398925781,
 269.3485412597656,
 269.347412109375,
 269.34625244140625,
 269.3450622558594,
 269.3438415527

In [19]:
#Turning array into series
dof = fc_day_array.date["date.dayofyear"].to_series() #<--day forecast was made(8)
for_day = fc_day_array.step["step.days"].to_series()#<--# days forecasted out(47)
day_series = fc_day_array.to_series()

#Turning Series into Lists
dof_list = dof.tolist()
for_day_list = for_day.tolist()
day_list = day_series.tolist()

In [20]:
#trying to create a series of the climo temps for each forcast day in day_list
climo_temp_series = pd.DataFrame()

for i in day_list:
    if day_list[i]==ct_time_list:
        climo_temp = ct_series.loc('time')
        climo_temp_series.append(columns =['forcast_day', 'climo_temp'])


In [22]:
climo_temp_series

""


In [ ]:
#Not working, uses too much memory
anom_t2 = pd.DataFrame()

for i in dof:
    for j in for_day:
        for time in ct_0z:
            if (step==time).all():
                dif = t_ensmn - ct_0z
                anom_t2.append(columns =['forcast_day', 'step', 'dif'])
#### MemoryError: Unable to allocate 167. GiB for an array with shape (47, 121, 240, 256, 512) and data type bool

In [ ]:
#No error, but produces nothing... :C
anom_t = pd.DataFrame()

for i in forcast_day:
    for step in t_ensmn:
        for time in ct_0z:
            if (step==day).all():
                dif = t_ensmn - ct_0z
                anom_t.append(columns =['forcast_day', 'step', 'dif'])
   

In [ ]:
#day_pds = pd.DataFrame()
#
#for i in dof:
#    for k in for_day:
#        day = i + k
#        day_pds.append(day)

In [ ]:
#Proper process. Need to figure out proper notation. May need to change data into pandas.
#dif[]
#for day S2S:
#    for step S2S:
#        for day climo:
#            if day==step:
#                dif = temp[s2s]-temp[climo]
#                dif.append(day,step,dif)

xesmf will help with gridding

write values to netcdf

DON'T CONVERT XARRAY TO NUMPY